# Libraries

In [1]:
import pandas as pd   # for parsing query into dataframe
import time

# Initialise Client API

Indicate the project used for making query

In [2]:
from google.cloud import bigquery
client = bigquery.Client(project="treenyc-384104") # mark the created project id

# Explore Public Datasets

In [ ]:
# this will get the names and descriptions of public datasets 
with open("meta/public_ds.txt", 'w') as f:
    for ds in client.list_datasets(project='bigquery-public-data'):
        f.write(ds.dataset_id + ": " + str(client.get_dataset(ds.reference).description)[:100] + '\n')
    f.close()

In [ ]:
def get_tables(dataset, client = client):
    """get table names and schema from given public dataset name"""
    for table in client.list_tables(f"bigquery-public-data.{dataset}"):
        table = client.get_table(table)
        print(table.full_table_id)
        print([item.name for item in table.schema])

    return

In [ ]:
get_tables("geo_us_roads")

bigquery-public-data:geo_us_roads.all_roads_01
['road_geom', 'road_id', 'full_name', 'route_type', 'mtfcc_feature_class_code']
bigquery-public-data:geo_us_roads.all_roads_02
['road_geom', 'road_id', 'full_name', 'route_type', 'mtfcc_feature_class_code']
bigquery-public-data:geo_us_roads.all_roads_04
['road_geom', 'road_id', 'full_name', 'route_type', 'mtfcc_feature_class_code']
bigquery-public-data:geo_us_roads.all_roads_05
['road_geom', 'road_id', 'full_name', 'route_type', 'mtfcc_feature_class_code']
bigquery-public-data:geo_us_roads.all_roads_06
['road_geom', 'road_id', 'full_name', 'route_type', 'mtfcc_feature_class_code']
bigquery-public-data:geo_us_roads.all_roads_08
['road_geom', 'road_id', 'full_name', 'route_type', 'mtfcc_feature_class_code']
bigquery-public-data:geo_us_roads.all_roads_09
['road_geom', 'road_id', 'full_name', 'route_type', 'mtfcc_feature_class_code']
bigquery-public-data:geo_us_roads.all_roads_10
['road_geom', 'road_id', 'full_name', 'route_type', 'mtfcc_featu

KeyboardInterrupt: 

# Retrieve Schema

Retrieve schemas for each tables to check compatability.

In [3]:
# get tables name inside dataset
table_names = [table.table_id for table in client.list_tables("bigquery-public-data.new_york_trees")]
print(table_names)

['tree_census_1995', 'tree_census_2005', 'tree_census_2015', 'tree_species']


In [3]:
descrption_dfs = {}
for table in client.list_tables("bigquery-public-data.new_york_trees"):
    table_schema = client.get_table(table).schema
    table_metadata = pd.DataFrame(
        {
            "Name" : [item.name for item in table_schema],
            "Dtype" : [item.field_type for item in table_schema],
            "Example" : client.list_rows(table=table, max_results=1).to_dataframe().values.flatten(),
            "Description":[item.description for item in table_schema],
        }
    )
    with open(f"meta/{table.table_id}_meta.txt", 'w') as f:
        f.write(table_metadata.to_markdown())
        f.close()
    descrption_dfs[table.table_id] = table_metadata

Count number of rows in tables

In [4]:
for table in client.list_tables("bigquery-public-data.new_york_trees"):
    table = client.get_table(table)
    print(f"{table.table_id}: {table.num_rows}")

tree_census_1995: 516989
tree_census_2005: 592372
tree_census_2015: 683788
tree_species: 57


# Query

In [31]:
def query_to_df(query_script, client=client):
    """pass a SQL query, return as pandas dataframe"""
    start = time.time()
    query = client.query(query_script) # create query
    df = query.to_dataframe()   # convert into pandas dataframe
    print(f"Query Time...{time.time()-start:.3}")  # retrieve query time
    print(df.info())

    return df

Write the query using SQL

In [40]:
QUERY = (   # query in SQL style
    """
    SELECT *
    FROM `bigquery-public-data.new_york_trees.tree_census_2005`
    LIMIT 100
    """
)

Run the main query and get as a pandas dataframe

In [41]:
df = query_to_df(QUERY)

Query Time...1.92
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 54 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   objectid      100 non-null    Int64  
 1   cen_year      100 non-null    Int64  
 2   tree_dbh      100 non-null    Int64  
 3   tree_loc      100 non-null    object 
 4   pit_type      100 non-null    object 
 5   soil_lvl      100 non-null    object 
 6   status        100 non-null    object 
 7   spc_latin     100 non-null    object 
 8   spc_common    100 non-null    object 
 9   vert_other    100 non-null    boolean
 10  vert_pgrd     100 non-null    boolean
 11  vert_tgrd     100 non-null    boolean
 12  vert_wall     100 non-null    boolean
 13  horz_blck     100 non-null    boolean
 14  horz_grate    100 non-null    boolean
 15  horz_plant    100 non-null    boolean
 16  horz_other    100 non-null    boolean
 17  sidw_crack    100 non-null    boolean
 18  sidw_raise   

# EDA

Explroe with variables of `tree_species`

In [36]:
tree_species = df.copy()
for column in tree_species.columns:
    print(column, ":", "\n",  tree_species[column].nunique(), tree_species[column].unique()[:5])

species_scientific_name : 
 57 ['Quercus phellos' 'Cotinus sp.' 'Prunus sargentii' 'Prunus padus'
 "Prunus serrulata 'Kwanzan'"]
species_common_name : 
 57 ['Willow Oak' 'Smoke Tree' 'Sargent Cherry' 'European Birdcherry'
 'Japanese Flowering Cherry']
form : 
 4 ['Rounded' 'Upright' 'Pyramidal' 'Vase–Like']
growth_rate : 
 2 ['Slow' 'Medium']
fall_color : 
 13 ['Yellow/Orange' 'Red/Bronze' 'Yellow/Bronze' 'Red/Orange' 'Red']
environmental_tolerances : 
 30 ['Drought and Pollution Tolerant' 'none' 'Shade Tolerant'
 'High pH Tolerant' 'Wet Site and Salt Tolerant']
location_tolerances : 
 8 ['Median Tree' 'Small Tree Pit (<3 ft)'
 'Median Tree, Small Tree Pit (<3 ft)'
 'Small Tree Pit (<3 ft), Narrow Growing Space' 'none']
notes_suggested_cultivars : 
 45 ['Plant Spring Only' "'Royal Purple' or 'Grace'"
 "'Accolade' is Semi–double Flowering;"
 'One of the First Trees to Leaf Out in the Spring' 'Double–flowering']
tree_size : 
 4 ['Large (Mature Height > 50 ft)' 'Small (Mature Height < 25 

In [64]:
tree_2005 = df.copy()
for column in tree_2005.columns:
    print(column, ":", tree_2005[column].nunique(), "==>", tree_2005[column].unique().tolist()[:5])

objectid : 100 ==> [1091298, 1091299, 1091300, 1091301, 1092323]
cen_year : 1 ==> [0]
tree_dbh : 21 ==> [7, 6, 5, 20, 10]
tree_loc : 5 ==> ['Front', 'Side', 'Assigned', 'Median', 'Across']
pit_type : 3 ==> ['Lawn', 'Continuous Pit', 'Sidewalk Pit']
soil_lvl : 2 ==> ['Level', 'Above']
status : 4 ==> ['Good', 'Excellent', 'Poor', 'Dead']
spc_latin : 17 ==> ['UNKNOWN', 'GLEDITSIA TRIACANTHOS', 'PYRUS CALLERYANA', 'QUERCUS PALUSTRIS', 'LIQUIDAMBAR STYRACIFLUA']
spc_common : 17 ==> ['UNKNOWN', 'HONEYLOCUST', 'PEAR, CALLERY', 'OAK, PIN', 'SWEETGUM']
vert_other : 1 ==> [False]
vert_pgrd : 1 ==> [False]
vert_tgrd : 2 ==> [False, True]
vert_wall : 1 ==> [False]
horz_blck : 2 ==> [False, True]
horz_grate : 1 ==> [False]
horz_plant : 2 ==> [False, True]
horz_other : 1 ==> [False]
sidw_crack : 2 ==> [False, True]
sidw_raise : 2 ==> [False, True]
wire_htap : 1 ==> [False]
wire_prime : 1 ==> [False]
wire_2nd : 1 ==> [False]
wire_other : 1 ==> [False]
inf_canopy : 1 ==> [False]
inf_guard : 1 ==> [Fal